In [49]:
import easydict
import os
import time

import torch
import torch.utils.data
import torch.utils.tensorboard
import torch.nn as nn
#import tqdm

# ipynb파일을 import할 수 있게 해주는 모듈
# pip install import_ipynb
import import_ipynb
import YOLOv3

In [47]:
def parse_data_config(path: str):
    print(path)
    """데이터셋 설정 파일 분석"""
    options = {}
    with open(path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options

def load_classes(path: str):
    print(path)
    """클래스 이름 로드"""
    with open(path, "r") as f:
        names = f.readlines()
    for i, name in enumerate(names):
        names[i] = name.strip()
    return names

def init_weights_normal(m):
    """정규분포 형태로 가중치 초기화"""
    classname = m.__class__.__name__
    # https://discuss.pytorch.org/t/object-has-no-attribute-weight/31526
    # if classname.find("Conv") != -1:
    if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight.data, 0.1)

    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


In [51]:
# 다양한 입력 매개변수를 분석한다. 아니면 기본 매개변수를 사용한다
# ipynb에서는 argparse를 사용할 수 없다. easydict로 대체할 쉬 있다. 
# https://worthpreading.tistory.com/56
args = easydict.EasyDict({
    "epoch": 20,
    "gradient_accumulation": 1,
    "multiscale_training": True,
    "batch_size": 32,
    "num_workers": 8,
    "data_config": "config/coco.data",
    "pretrained_weights": 'weights/darknet53.conv.74',
    "image_size": 416
})
print(args)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
now = time.strftime('%y%m%d_%H%M%S', time.localtime(time.time()))

# Tensorboard writer 객체 생성
log_dir = os.path.join('logs', now)
os.makedirs(log_dir, exist_ok=True)
writer = torch.utils.tensorboard.SummaryWriter(log_dir)

# 데이터셋 설정값을 가져오기
data_config = parse_data_config(args.data_config)
train_path = data_config['train']
valid_path = data_config['valid']
num_classes = int(data_config['classes'])
class_names = load_classes(data_config['names'])


# 모델 준비하기
model = YOLOv3.YOLOv3(args.image_size, num_classes).to(device)
model.apply(init_weights_normal)
if args.pretrained_weights.endswith('.pth'):
    model.load_state_dict(torch.load(args.pretrained_weights))
else:
    model.load_darknet_weights(args.pretrained_weights)


{'epoch': 20, 'gradient_accumulation': 1, 'multiscale_training': True, 'batch_size': 32, 'num_workers': 8, 'data_config': 'config/coco.data', 'pretrained_weights': 'weights/darknet53.conv.74', 'image_size': 416}
config/coco.data
data/coco/coco_classes.txt


ModuleAttributeError: 'YOLOv3' object has no attribute 'load_darknet_weights'